In [1]:
import pandas as pd
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from keras.models import Sequential 
from keras.layers import Dense,Activation,Dropout 
from random import random
import numpy as np
from sklearn import preprocessing
import math
iris=pd.read_csv("Iris.csv")

In [2]:
def normalize(dataset):
    for column in dataset.columns:
        dataset[column]=(dataset[column]-min(dataset[column]))/(max(dataset[column])-min(dataset[column]))
    return dataset
data=normalize(iris.loc[:,iris.columns[:-1]])
print(data)

     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0         0.222222      0.625000       0.067797      0.041667
1         0.166667      0.416667       0.067797      0.041667
2         0.111111      0.500000       0.050847      0.041667
3         0.083333      0.458333       0.084746      0.041667
4         0.194444      0.666667       0.067797      0.041667
..             ...           ...            ...           ...
145       0.666667      0.416667       0.711864      0.916667
146       0.555556      0.208333       0.677966      0.750000
147       0.611111      0.416667       0.711864      0.791667
148       0.527778      0.583333       0.745763      0.916667
149       0.444444      0.416667       0.694915      0.708333

[150 rows x 4 columns]


In [3]:
label_encoder = preprocessing.LabelEncoder()
iris['species']= label_encoder.fit_transform(iris['Species'])
data=pd.concat([data,iris["species"]],axis=True)

In [4]:
def train_test(data):
    x_train,x_test,y_train,y_test=train_test_split(data.loc[:,data.columns[:-1]], data.loc[:,data.columns[-1]],test_size=0.33, random_state=42)
    return  x_train,x_test,y_train,y_test
x_train,x_test,y_train,y_test=train_test(data)

In [5]:
y_train.to_numpy()

array([1, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0,
       2, 2, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 2,
       2, 1, 0, 0, 2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2,
       1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2,
       1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2])

In [6]:
def initaial_network(n_inputs,n_outputs,neuron,x):
    network=list()
    hidden_layer={"weights":[[random() for j in range(neuron)]]*len(x),"bias":[[1 for j in range(neuron)]]*len(x)}
    output_layer={"weights":[[random() for j in range(n_outputs)]]*len(x),"bias":[[1 for j in range(n_outputs)]]*len(x)}
    network.append(hidden_layer)
    network.append(output_layer)
    return network

In [7]:
def activation(k,k1,net):
    return k+k1*net
def softmax(a):
    e=[math.exp(i) for i in a]
    exp=[j/sum(e) for j in e]
    return exp

In [8]:
x=[[1,1],[1,1]]

In [9]:
def update(dw2,db2,dw1,db1,network):
    for i in range(len(network)-1):
        for j in range(len(network[i]["weights"])):
            n=[]
            b=[]
            for k in range(len(network[i]["weights"][j])-1):
                n.append((network[i]["weights"][j][k]-0.01*dw1[k])[0])
                b.append(network[i]["bias"][j][k]-0.01*db1)
            network[i]["weights"][j]=n
            network[i]["bias"][j]=b
    for j in range(len(network[-1]["weights"])):
            n=[]
            b=[]
            for k in range(len(network[-1]["weights"][j])-1):
                n.append((network[-1]["weights"][j][k]-0.01*dw2[k])[0])
                b.append(network[-1]["bias"][j][k]-0.01*db2)
            network[-1]["weights"][j]=n
            network[-1]["bias"][j]=b
    return network
            

def back_p(y_train,Z,A,network,output,K):
    Z=np.array(Z)
    dz2=(np.array(output)-np.array(y_train)).reshape(len(y_train),1)
    dw2=np.matmul((np.array(A[-1]).T)/len(A[-1]),dz2)
    db2=np.average(dz2)
    w=np.array(network[-1]["weights"])
    da1=np.matmul(w.T,dz2)
    dz1=K[1]*np.array(A[1])
    dw1=np.matmul(np.array(A[0]).T/len(A[0]),dz1)
    db1=np.average(dz1)
    k=[np.average(da1),np.average(Z[0])]
    network=update(dw2,db2,dw1,db1,network)
    return k,network
def accuracy_met(actual, predicted):
        correct = 0
        for i in range(len(actual)):
                if actual[i] == predicted[i]:
                        correct += 1
        return correct / float(len(actual)) * 100.0
    
def train(x_train,y_train,network,epochs):
    k,k1=random(),random()
    K=[k,k1]
    for j in range(epochs):
        print(j)
        output=[]
        output_values=[]
        Z=[]
        A=[x_train]
        A_iter=[]
        Z_iter=[]
        for row in range(len(x_train)):
            inputs=x_train[row]
            new_a=[]
            new_z=[]
            for layer in range(len(network)-1):
                for neuron,bias in zip(network[layer]["weights"][row],network[layer]["bias"][row]):
                    z=bias
                    for value in inputs:
                        z+=value*neuron
                    a=activation(z,k,k1)
                    new_z.append(z)
                    new_a.append(a)
                inputs=new_a
            Z_iter.append(new_z)
            A_iter.append(new_a)
            a=[]
            for neuron,bias in zip(network[-1]["weights"][row],network[-1]["bias"][row]):
                z=bias
                for value in inputs:
                    z+=value*neuron
                a.append(z)
            a=softmax(a)
            output_values.append(a)
            output.append(a.index(max(a)))
        Z.append(Z_iter)
        A.append(A_iter)
        A_iter=[]
        Z_iter=[]
        K,network=back_p(y_train,Z,A,network,output,K)
        k=K[0]
        k1=K[1]
        pred=accuracy_met(y_train,output)
    return pred

In [10]:
network=initaial_network(4,2,5,x_train.to_numpy())
train(x_train.to_numpy(),y_train.to_numpy(),network,1)

0


35.0